In [1]:
import sys
sys.path.append('../')
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.fx import subgraph_rewriter, symbolic_trace
import utils
from torch.fx import Proxy, Graph, GraphModule
from torch.fx.passes.utils.matcher_utils import SubgraphMatcher
import onn

replace success!


In [2]:
onn_model = onn.ONN([100 for i in range(22)],8,[400,400,400],0.1)
traced = symbolic_trace(onn_model)

In [3]:
%timeit -n 1 -r 30 with torch.no_grad(): output = onn_model(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

23.1 ms ± 2.83 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [4]:
interp = utils.ProfilingInterpreter(onn_model)
interp.run(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))
print(interp.summary(True))

total true time 42.513132095336914 ms
total time: 45.94850540161133 ms
Op type        Op               Average runtime (ms)    Pct total runtime
-------------  -------------  ----------------------  -------------------
call_module    mlp_mlp_2                   1.04189              2.26751
call_function  cat                         0.909567             1.97954
call_module    mlp_mlp_0                   0.818729             1.78184
call_module    mlp_mlp_6                   0.778913             1.69519
call_module    mlp_mlp_3                   0.774622             1.68585
call_module    mlp_mlp_5                   0.73719              1.60438
call_module    mlp_mlp_8                   0.688314             1.49801
call_function  cat_1                       0.669718             1.45754
call_function  getitem_371                 0.577927             1.25777
call_module    embeddings_0                0.4282               0.931912
call_module    embeddings_21               0.419378         

In [5]:
onn_model = onn.ONN([100 for i in range(100)],32,[400,400,400],0.1)

In [6]:
interp = utils.ProfilingInterpreter(onn_model)
interp.run(torch.randint(low=0, high=88, size=(1024,100), dtype=torch.long))
print(interp.summary(True))

total true time 1331.4111232757568 ms
total time: 1412.8429889678955 ms
Op type        Op               Average runtime (ms)    Pct total runtime
-------------  -------------  ----------------------  -------------------
call_function  cat                        26.4218              1.87011
call_module    mlp_mlp_0                   5.56755             0.394067
call_function  cat_1                       3.76344             0.266373
call_module    embeddings_6                1.82152             0.128926
call_module    embeddings_4                1.81246             0.128284
call_module    embeddings_5                1.7879              0.126546
call_module    embeddings_7                1.42765             0.101048
call_function  getitem_8344                1.41668             0.100272
call_function  getitem_1324                1.1313              0.0800723
call_module    mlp_mlp_6                   1.03617             0.0733392
call_module    embeddings_0                1.02949         

In [2]:
onn_model = onn.ONNV2([100 for i in range(100)],32,[400,400,400],0.1)

In [3]:
output = onn_model(torch.randint(low=0, high=88, size=(1024,100), dtype=torch.long))

In [4]:
interp = utils.ProfilingInterpreter(onn_model)
interp.run(torch.randint(low=0, high=88, size=(1024,100), dtype=torch.long))
print(interp.summary(True))

total true time 2065.7622814178467 ms
total time: 2130.3281784057617 ms
Op type        Op                     Average runtime (ms)    Pct total runtime
-------------  -------------------  ----------------------  -------------------
call_function  masked_select                  1816.69              85.2774
call_module    embedding_embedding              77.8666             3.65514
call_function  mul                              64.3175             3.01913
call_function  masked_select_1                  63.3028             2.9715
call_method    sum_1                            32.989              1.54854
call_module    mlp_mlp_0                         4.3416             0.2038
call_function  cat                               2.56252            0.120288
call_module    mlp_mlp_6                         0.768423           0.0360706
call_module    mlp_mlp_3                         0.402451           0.0188915
call_function  add                               0.352383           0.0165412
call

In [12]:
torch.onnx.export(onn_model,torch.randint(low=0, high=88, size=(1024,100), dtype=torch.long),'onn.onnx',input_names = ['input'],)

In [19]:
%timeit -n 1 -r 30 with torch.no_grad(): output = onn_model(torch.randint(low=0, high=88, size=(1024,100), dtype=torch.long))

2.18 s ± 42.2 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [7]:
import onnx
onnx_model = onnx.load('onn.onnx')

In [9]:
import onnxruntime as ort
import numpy as np


In [15]:
session = ort.InferenceSession("onn.onnx")


In [17]:
%timeit -n 1 -r 30 session.run(None,{'input' : np.random.randint(low=0, high=88, size=(1024, 100), dtype=np.int64)})

170 ms ± 58.1 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [20]:
class FieldAwareFactorizationMachine(torch.nn.Module):

    def __init__(self, field_dims, embed_dim):
        super().__init__()
        self.num_fields = len(field_dims)
        self.embeddings = torch.nn.ModuleList([
            torch.nn.Embedding(sum(field_dims), embed_dim) for _ in range(self.num_fields)
        ])
        # self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.long)
        self.offsets = torch.as_tensor(np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.int64))

        for embedding in self.embeddings:
            torch.nn.init.xavier_uniform_(embedding.weight.data)
    def ffm_interaction(self,field_wise_emb_list):
        interaction = []
        num_fields = self.num_fields
        for i in range(num_fields - 1):
            for j in range(i + 1, num_fields):
                v_ij = field_wise_emb_list[j - 1][:, i, :]
                v_ji = field_wise_emb_list[i][:, j, :]
                dot = torch.sum(v_ij * v_ji, dim=1, keepdim=True)
                interaction.append(dot)
        return torch.cat(interaction, dim=1)
    def forward(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        # x = x + x.new_tensor(self.offsets).unsqueeze(0)
        x = x + self.offsets

        xs = [self.embeddings[i](x) for i in range(self.num_fields)]
        return self.ffm_interaction(xs)

In [21]:
ffm_model = FieldAwareFactorizationMachine([10 for _ in range(8)],8)

In [22]:
ffm_model_traced = symbolic_trace(ffm_model)

In [23]:
ffm_model_traced.print_readable()

class FieldAwareFactorizationMachine(torch.nn.Module):
    def forward(self, x):
        # No stacktrace found for following nodes
        offsets = self.offsets
        add = x + offsets;  x = offsets = None
        embeddings_0 = getattr(self.embeddings, "0")(add)
        embeddings_1 = getattr(self.embeddings, "1")(add)
        embeddings_2 = getattr(self.embeddings, "2")(add)
        embeddings_3 = getattr(self.embeddings, "3")(add)
        embeddings_4 = getattr(self.embeddings, "4")(add)
        embeddings_5 = getattr(self.embeddings, "5")(add)
        embeddings_6 = getattr(self.embeddings, "6")(add)
        embeddings_7 = getattr(self.embeddings, "7")(add);  add = None
        getitem = embeddings_0[(slice(None, None, None), 0, slice(None, None, None))]
        getitem_1 = embeddings_0[(slice(None, None, None), 1, slice(None, None, None))]
        mul = getitem * getitem_1;  getitem = getitem_1 = None
        sum_1 = torch.sum(mul, dim = 1, keepdim = True);  mul = None
        

'class FieldAwareFactorizationMachine(torch.nn.Module):\n    def forward(self, x):\n        # No stacktrace found for following nodes\n        offsets = self.offsets\n        add = x + offsets;  x = offsets = None\n        embeddings_0 = getattr(self.embeddings, "0")(add)\n        embeddings_1 = getattr(self.embeddings, "1")(add)\n        embeddings_2 = getattr(self.embeddings, "2")(add)\n        embeddings_3 = getattr(self.embeddings, "3")(add)\n        embeddings_4 = getattr(self.embeddings, "4")(add)\n        embeddings_5 = getattr(self.embeddings, "5")(add)\n        embeddings_6 = getattr(self.embeddings, "6")(add)\n        embeddings_7 = getattr(self.embeddings, "7")(add);  add = None\n        getitem = embeddings_0[(slice(None, None, None), 0, slice(None, None, None))]\n        getitem_1 = embeddings_0[(slice(None, None, None), 1, slice(None, None, None))]\n        mul = getitem * getitem_1;  getitem = getitem_1 = None\n        sum_1 = torch.sum(mul, dim = 1, keepdim = True);  mu